In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 引入库

In [ ]:
import numpy as np
import pandas as pd
import datatable as dt
import psutil
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
import riiideducation
from pandas_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot as plt 
%matplotlib inline
import seaborn as sns

# 读入数据

In [ ]:
train_df = pd.DataFrame()
counter = 1
for chunk in pd.read_csv('../input/riiid-test-answer-prediction/train.csv', chunksize=1000000, low_memory=False, nrows=10000000):
    print('Reading chunck {}'.format(counter))
    train_df = pd.concat([train_df, chunk], ignore_index=True)
    counter += 1

In [ ]:
test_df = pd.read_csv('../input/riiid-test-answer-prediction/example_test.csv')
questions_df = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv')
lectures_df = pd.read_csv('../input/riiid-test-answer-prediction/lectures.csv')

# 分析数据-train_df

In [ ]:
train_df

In [ ]:
print(train_df['answered_correctly'].isna().sum())
print(train_df.shape[0])
print('There are {:0.4f}% of missing data in answered_correctly. '.format(train_df['answered_correctly'].isna().sum()/train_df.shape[0]))

In [ ]:
print(train_df['prior_question_elapsed_time'].isna().sum())
print(train_df.shape[0])
print('There are {:0.4f}% of missing data in prior_question_elapsed_time. '.format(train_df['prior_question_elapsed_time'].isna().sum()/train_df.shape[0]))

In [ ]:
print(train_df['prior_question_had_explanation'].isna().sum())
print(train_df.shape[0])
print('There are {:0.4f}% of missing data in prior_question_had_explanation. '.format(train_df['prior_question_had_explanation'].isna().sum()/train_df.shape[0]))

### 填补Nan

In [ ]:
# Code from https://www.kaggle.com/dmikar/baseline-for-riiid-lightgbm
mean_prior = train_df['prior_question_elapsed_time'].astype("float64").mean()
print(f'{mean_prior} is filled for the missing data in prior_question_elapsed_time. ')

train_df['prior_question_elapsed_time'].fillna(mean_prior, inplace = True)
train_df['prior_question_had_explanation'].fillna(False, inplace = True)

# 分析数据-test_df

In [ ]:
test_df

In [ ]:
test_df['prior_question_had_explanation'].dtype

# 分析数据-questions_df

In [ ]:
questions_df

In [ ]:
questions_df['tag'] = questions_df['tags'].str.split()

In [ ]:
questions_df

In [ ]:
print(f"There are {train_df['user_id'].unique().shape[0]} users in training data.") 
print(f"There are {test_df['user_id'].unique().shape[0]} users in testing data.") 
total_users = pd.concat([train_df['user_id'], test_df['user_id']])
print(f"There are {total_users.unique().shape[0]} users in either data frames." )

In [ ]:
y = train_df.loc[0:1000000,['answered_correctly']]
X = train_df.loc[0:1000000,['task_container_id', 'content_type_id','prior_question_elapsed_time', 'prior_question_had_explanation']]

In [ ]:
mini_max_scaler = MinMaxScaler()
X = mini_max_scaler.fit_transform(X)


In [ ]:
X_train_df, X_val_df, y_train_df, y_val_df = train_test_split(X, y, test_size =0.3, shuffle=False)

In [ ]:
# X_train_df['prior_question_had_explanation'].isna().sum()

In [ ]:
X_train_df = X_train_df.astype('float64')
X_val_df = X_val_df.astype('float64')
y_train_df = y_train_df.astype('float64')
y_val_df = y_val_df.astype('float64')

In [ ]:
X_train_df

In [ ]:
from keras import Sequential
from keras.layers import Dense
import lightgbm as lgbm
# 定义模型
model=lgbm.LGBMClassifier(num_leaves=60,learning_rate=0.05,n_estimators=40)
model.fit(X_train_df,y_train_df)
y_pre=model.predict(X_val_df)
county = 0
count = 0
y_val_df = y_val_df.values
print(y_val_df)
for i in range(len(y_pre)):
    count+=1
    if(y_pre[i] == y_val_df[i]):
        county+=1
print(county/count)

In [ ]:
# model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
# nb_epoch = 5
# model.fit(X_train_df, y_train_df, epochs=nb_epoch, validation_data=(X_val_df, y_val_df), batch_size=16)

In [ ]:
ex = pd.read_csv('../input/riiid-test-answer-prediction/example_test.csv')
ex = ex[ex['content_type_id'] == 0]
ex1 = ex[['user_id', 'content_type_id', 'prior_question_elapsed_time', 'prior_question_had_explanation']]
# 这里做你的处理
mean_prior_test = ex1['prior_question_elapsed_time'].astype("float64").mean()
ex1['prior_question_elapsed_time'].fillna(mean_prior_test, inplace=True)
ex1['prior_question_had_explanation'].fillna(False, inplace=True)
ex1 = ex1.astype('float64')
ex1 = mini_max_scaler.fit_transform(ex1)
print(ex1)

In [ ]:
res = model.predict(ex1)
ex['answered_correctly']=res
# res = sum(res, [])
ress = ex[['row_id', 'answered_correctly']]
print(ress)

In [ ]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
for (sample_test,sample_prediction_df) in iter_test:
    sample_test = sample_test[sample_test['content_type_id'] == 0]
    sample_test1 = sample_test[['user_id', 'content_type_id', 'prior_question_elapsed_time', 'prior_question_had_explanation']]
    # 这里做你的处理
    mean_prior_test = sample_test1['prior_question_elapsed_time'].astype("float64").mean()
    sample_test1['prior_question_elapsed_time'].fillna(mean_prior_test, inplace=True)
    sample_test1['prior_question_had_explanation'].fillna(False, inplace=True)
    sample_test1 = sample_test1.astype('float64')
    sample_test1 = mini_max_scaler.fit_transform(sample_test1)
    
    res = model.predict(sample_test1)
    sample_test['answered_correctly']=res
    
    env.predict(sample_test[['row_id','answered_correctly']])